In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"
import time
import random
import glob
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.distributed as dist
from torch.utils.data import Dataset, DataLoader, DistributedSampler
from torch.multiprocessing.spawn import spawn
import torchvision.models as models
import torchvision.datasets as dst
from torchvision.io import read_image
from torchvision.transforms import v2

In [ ]:
# dataset
DATA_LOOT = "/export/data/dataset/COCO/train2014"
STYLE_IMG_PATH = "./data/NST/gohho.png"
DATA_SIZE = 128

class CustomDataset(Dataset):
    def __init__(self, data_root, style_img_path, img_size):
        self.content_paths = glob.glob(os.path.join(data_root, "*.jpg"))

        self.transforms = v2.Compose([
            v2.Resize(img_size),
            v2.CenterCrop(img_size),
            v2.ToTensor(),
            v2.Lambda(lambda x: x.mul(255)), #VGGは256スケールで学習
            v2.Normalize(
                mean=[0.485, 0.456, 0.406],
                std =[0.229, 0.224, 0.225]
            )
        ])

        # スタイル画像読み込み
        style_img = read_image(style_img_path)
        self.style_tensor = self.transforms(style_img)

    def __len(self):
        return len(self.content_paths)

    def __getitem__(self, idx):
        content_img = read_image(self.content_paths[idx])
        content_tensor = self.transforms(content_img)
        return content_tensor, self.style_tensor

SyntaxError: invalid syntax (3498195004.py, line 14)

In [10]:
imgs = glob.glob(os.path.join(DATA_LOOT, "*.jpg"))
len(imgs)

82783